In [1]:
!python --version

Python 3.11.11


In [2]:
!pip freeze > '/kaggle/working/requirements.txt'

In [3]:
!pip install evaluate rouge_score bitsandbytes rouge --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependenc

In [4]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, GenerationConfig, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType

2025-05-15 05:02:31.968761: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747285352.191969      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747285352.266617      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
parameters = {
    'ENCODER_SEQUENCE_LENGTH': 512,
    'DECODER_SEQUENCE_LENGTH': 26,
    'EPOCHS': 1,
    'L2_REGULARIZATION': 0.01,
    'BATCH_SIZE': 16,
    'LEARNING_RATE': 2e-5,
    'LORA_R': 16,
    'LORA_ALPHA': 32,
    'LORA_DROPOUT': 0.1,
    'WARMUP_STEPS': 1_000
}

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4",
)

In [7]:
tokenizer = MBart50TokenizerFast.from_pretrained('facebook/mbart-large-50')
model = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50', quantization_config=bnb_config)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [8]:
tokenizer.src_lang = "ne_NP"
tokenizer.tgt_lang = "ne_NP"
model.generation_config = GenerationConfig.from_pretrained('facebook/mbart-large-50')
# model.generation_config.forced_bos_token_id = tokenizer.lang_code_to_id["ne_NP"]

In [9]:
lora_config = LoraConfig(
    r=parameters['LORA_R'],
    lora_alpha=parameters['LORA_ALPHA'],
    lora_dropout=parameters['LORA_DROPOUT'],
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,359,296 || all params: 613,238,784 || trainable%: 0.3847


In [10]:
train_file = '/kaggle/input/llm-summarization-set-cleaned/llm_summarization_set_cleaned_train.csv'
test_file = '/kaggle/input/llm-summarization-set-cleaned/llm_summarization_set_cleaned_test.csv'
dataset = load_dataset("csv", data_files={"train": train_file, "test": test_file})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [11]:
train = dataset['train']
test = dataset['test']

In [12]:
def tokenize(data):
    inputs = tokenizer(data["news"], truncation=True, padding="max_length", max_length=parameters['ENCODER_SEQUENCE_LENGTH'])
    targets = tokenizer(text_target=data["title"], truncation=True, padding="max_length", max_length=parameters['DECODER_SEQUENCE_LENGTH'])
    inputs["labels"] = targets["input_ids"]
    return inputs

In [13]:
tokenized_train_dataset = train.map(tokenize, batched=True, remove_columns=['news', 'title'])
tokenized_test_dataset = test.map(tokenize, batched=True, remove_columns=['news', 'title'])

Map:   0%|          | 0/300371 [00:00<?, ? examples/s]

Map:   0%|          | 0/15810 [00:00<?, ? examples/s]

In [14]:
args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/tmp",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=parameters['LEARNING_RATE'],
    per_device_train_batch_size=parameters['BATCH_SIZE'],
    per_device_eval_batch_size=parameters['BATCH_SIZE'],
    weight_decay=parameters['L2_REGULARIZATION'],
    save_total_limit=2,
    num_train_epochs=parameters['EPOCHS'],
    warmup_steps=parameters['WARMUP_STEPS'],
    metric_for_best_model="loss",
    greater_is_better=False,
    fp16=True,
    logging_dir='/kaggle/tmp/',
    logging_strategy='epoch',
    report_to='none',
    label_names=["labels"],
    generation_config=model.generation_config,
    predict_with_generate=True,
    generation_max_length=parameters['DECODER_SEQUENCE_LENGTH'],
)

In [15]:
import evaluate
import numpy as np
from rouge import Rouge

rouge = Rouge()
bleu = evaluate.load("bleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    results = rouge.get_scores(preds, labels, avg=True)
    results = {f'{key}-{metric}': round(metric_value, 4) for key, value in results.items() for metric, metric_value in value.items()}
    bleu_result = bleu.compute(predictions=preds, references=[[ref] for ref in labels])
    results["bleu"] = bleu_result["bleu"]

    return results

In [16]:
collator = DataCollatorForSeq2Seq(tokenizer, model)
collator.save_pretrained = lambda *args, **kwargs: None

In [17]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    processing_class=collator,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge-1-r,Rouge-1-p,Rouge-1-f,Rouge-2-r,Rouge-2-p,Rouge-2-f,Rouge-l-r,Rouge-l-p,Rouge-l-f,Bleu
1,5.002600,4.498979,0.404600,0.411400,0.399200,0.225300,0.229100,0.221400,0.390400,0.396700,0.385000,0.173609


TrainOutput(global_step=18774, training_loss=5.0025500692447, metrics={'train_runtime': 28755.0918, 'train_samples_per_second': 10.446, 'train_steps_per_second': 0.653, 'total_flos': 3.276484838739149e+17, 'train_loss': 5.0025500692447, 'epoch': 1.0})

In [19]:
import tarfile
with tarfile.open("/kaggle/working/model_outputs.tar.gz", "w:gz") as tar:
    tar.add("/kaggle/tmp", arcname="model_outputs")